# Test create_matchup + add sum except

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
from rasterio.transform import Affine

import os
from glob import glob

import histlib.box as box
import histlib.aviso as aviso
import histlib.cstes as cstes
#import histlib.diagnosis as diag
import histlib.erastar as eras

from dask.distributed import wait
from histlib.cstes import labels, zarr_dir

/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


In [2]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    from dask import config
    config.set({"distributed.comm.timeouts.connect": "200s"})
    cluster = PBSCluster(cores=5, processes=5, walltime='04:00:00')
    #cluster = PBSCluster(cores=20, processes=20, walltime='02:00:00')#8
    w = cluster.scale(jobs=10)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.218:8787/status,
Dashboard: http://10.148.0.218:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.218:46670,Workers: 0
Dashboard: http://10.148.0.218:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# BASIC MATCHUP

In [3]:

def add_ggx_attrs(ds_data):
    """Add gradient attributes
    Parameters
    ----------
    ds_data: dataset
            dataset containing colocalisations
    """
    listv = [l for l in list(ds_data.variables) if 'sla' in l]+['alti_mdt','alti_ocean_tide', 'alti_dac', 'alti_internal_tide']
    listv = [l for l in listv if 'gg' not in l]
    for v in listv :
        ds_data[v.replace('alti', 'alti_ggx')].attrs['comment'] = ds_data[v].attrs['comment']
        ds_data[v.replace('alti', 'alti_ggx')].attrs['units'] = r'$m.s^{-2}$'
        ds_data[v.replace('alti', 'alti_ggx')].attrs['long_name']= r'$g\partial_x$'+v.replace('alti_','')

def add_adt_to_ds_data(ds_data):
    """Adds adt to dataset from sla + mdt and manages attributes
    Parameters
    ----------
    ds_data: dataset
            dataset containing colocalisations
    """
    add_ggx_attrs(ds_data)
    ds_data = ds_data.rename({'drifter_acc_x':'drifter_acc_x_0', 'drifter_acc_y':'drifter_acc_y_0', 'drifter_coriolis_x':'drifter_coriolis_x_0', 'drifter_coriolis_y':'drifter_coriolis_y_0'})
    for sla in ['alti_ggx_sla_filtered','alti_ggx_sla_unfiltered','alti_ggx_sla_unfiltered_denoised','alti_ggx_sla_unfiltered_imf1']:
        ds_data[sla.replace('sla', 'adt')] = ds_data[sla] + ds_data.alti_ggx_mdt
        ds_data[sla.replace('sla', 'adt')].attrs['comment'] = ds_data[sla].attrs['comment']
        ds_data[sla.replace('sla', 'adt')].attrs['units'] = r'$m.s^{-2}$'
        ds_data[sla.replace('sla', 'adt')].attrs['long_name']= ds_data[sla].attrs['long_name'].replace('sla', 'adt')
    return ds_data

def change_obs_coords(ds,l) :
    """Change obs coordinates including the label
    Parameters
    ----------
    ds_data: dataset
            dataset containing colocalisations
    l : label, 
            in cstes.labels
    """
    o = np.array(ds['obs']).astype('U')
    L = np.full_like(o, l+'__')
    ob = xr.DataArray(np.char.add(L, o), dims='obs')
    ds = ds.drop('obs')
    ds = ds.assign_coords({'obs':ob})
    return ds
    
import histlib.stress_to_windterm as stw
_data_var = [
    "f",
    "box_theta_lon",
    "__site_matchup_indice",
    "box_theta_lat",
    "drifter_theta_lon",
    "drifter_theta_lat",
    "drifter_typebuoy",
    "alti___distance",
    "alti___time_difference",
    'alti_ggx_dac',
    'alti_ggx_internal_tide',
    'alti_ggx_mdt',
    'alti_ggx_ocean_tide',
    'alti_ggx_sla_filtered',
    'alti_ggx_sla_unfiltered',
    'alti_ggx_sla_unfiltered_denoised',
    #'alti_ggx_sla_unfiltered_imf1',
    'alti_ggx_adt_filtered',
    'alti_ggx_adt_unfiltered',
    'alti_ggx_adt_unfiltered_denoised',
    #'alti_ggx_adt_unfiltered_imf1',
    "drifter_vx",
    "drifter_vy",
    "drifter_acc_x_0",
    "drifter_acc_y_0",
    "drifter_coriolis_x_0",
    "drifter_coriolis_y_0",
    
]
_aviso_var = [
    "aviso_alti_matchup_ggx_adt",
    "aviso_alti_matchup_ggy_adt",
    "aviso_drifter_matchup_ggx_adt",
    "aviso_drifter_matchup_ggy_adt",
    "aviso_alti_matchup_ggx_sla",
    "aviso_alti_matchup_ggy_sla",
    "aviso_drifter_matchup_ggx_sla",
    "aviso_drifter_matchup_ggy_sla",
]
_stress_var = [
    "e5_alti_matchup_taue",
    "e5_alti_matchup_taun",
    "es_alti_matchup_taue",
    "es_alti_matchup_taun",
    "e5_drifter_matchup_taue",
    "e5_drifter_matchup_taun",
    "es_drifter_matchup_taue",
    "es_drifter_matchup_taun",
]


list_wd_srce_suffix = ["es", "e5"]
list_func = [stw.cst_rio_z0, stw.cst_rio_z15]
list_func_suffix = ["cstrio_z0", "cstrio_z15"]


In [4]:
def matchup_dataset_one(l):
    """Gathers colocalisation, erastar and aviso data at matchup point, include drogue status. Return the matchup dataset corresponding to l
    Parameters
    ----------
    l : label, 
            in cstes.labels
    """
    ds_data = xr.open_zarr(os.path.join(zarr_dir, f'{l}.zarr')).chunk({'obs':500, 'site_obs':-1})
    ds_data = ds_data.where(ds_data.alti___distance<2e5, drop=True).chunk({'obs':500, 'site_obs':-1})
    ds_data = add_adt_to_ds_data(ds_data)
    ds_aviso = xr.open_zarr(os.path.join(zarr_dir+'_ok','aviso', f'aviso_{l}.zarr')).chunk({'obs':500})
    ds_stress = xr.open_zarr(os.path.join(zarr_dir+'_ok','erastar', f'erastar_{l}.zarr')).chunk({'obs':500})
    
    # COLOCALIZATIONS DATA

    drogue_status = ds_data.time<ds_data.drifter_drogue_lost_date.mean('site_obs')
    ds_data = ds_data[_data_var]
    cc = [l for l in list(ds_data.coords)if l not in ['obs', 'box_x', 'box_y', 'alti_time_mid']]
    ds_data = ds_data.reset_coords(cc)
    idx = ds_data.__site_matchup_indice.astype(int).compute()
    _ds_data = ds_data.sel(site_obs=idx).isel( alti_time_mid=ds_data.dims['alti_time_mid']//2).drop(["alti_time_mid", "alti_x_mid", "alti_y_mid"])
    #_ds_data = compute_dsdatamatchup(ds_data)
    for v in _ds_data.variables:
        _ds_data[v].attrs = ds_data[v].attrs
    _ds_data['drogue_status'] = drogue_status.drop(['lon', 'lat', 'time']).assign_attrs({'long_name':'drogue status', 'description':'True if drogued, False if undrogued (day precision only)'})

    # AVISO
    _ds_aviso = ds_aviso[_aviso_var]

    #ERASTAR
    _ds_stress = ds_stress[_stress_var]

    # FOR IND PDFS
    _ds = xr.merge([_ds_data, _ds_aviso, _ds_stress])

    # COMPUTE WD TERM
    _ds = xr.merge(
        [
            _ds,
            stw.compute_wd_from_stress(
                _ds, list_wd_srce_suffix, list_func, list_func_suffix, False
            ),
        ]
    )
    # CLEANING : drop useless variables
    _ds = _ds.drop(
        _stress_var
        + [
            "f",
            "box_theta_lon",
            "box_theta_lat",
            "drifter_theta_lon",
            "drifter_theta_lat",
            "__site_matchup_indice",
        ]
    ).set_coords(["alti___distance", "alti___time_difference"])
    _ds = _ds.rename({v: v.replace("_matchup", "") for v in _ds})
    #_ds = change_obs_coords(_ds, l)  
    _ds = _ds.drop(['box_x', 'box_y']).set_coords(['lon', 'lat','time'])
    return _ds

In [228]:
l='gps_Jason-3_2020'
ds = matchup_dataset_one(l)
ds.chunk({'obs':500}).to_zarr(os.path.join(zarr_dir, f'matchup_{l}_test.zarr'), mode ='w')

Frozen({})


In [230]:
ds_ = ds.isel(obs=slice(0,10)).compute()

In [231]:
ds_

<xarray.Dataset>
Dimensions:                           (obs: 10)
Coordinates:
    alti___distance                   (obs) float64 9.718e+04 ... 3.642e+04
    alti___time_difference            (obs) float64 778.0 206.0 ... 1.292e+03
    lat                               (obs) float64 -41.18 -27.01 ... 52.3 30.65
    lon                               (obs) float64 139.0 -145.3 ... -24.16
    time                              (obs) datetime64[ns] 2020-01-02T21:00:0...
  * obs                               (obs) <U42 'gps_Jason-3_2020__0' ... 'g...
Data variables: (12/47)
    drifter_typebuoy                  (obs) object b'SVPB   ' ... b'SVPB   '
    alti_ggx_dac                      (obs) float64 0.0 -1.581e-06 ... 0.0
    alti_ggx_internal_tide            (obs) float64 0.0 3.162e-06 ... 0.0 0.0
    alti_ggx_mdt                      (obs) float64 -1.583e-06 ... 0.0
    alti_ggx_ocean_tide               (obs) float64 0.0 0.0 ... 4.752e-06 0.0
    alti_ggx_sla_filtered             (obs) float64 -3.165e-06 0.0 ... 0.0
    ...                                ...
    es_cstrio_z15_drifter_wd_x        (obs) float64 -5.052e-07 nan ... 1.701e-07
    es_cstrio_z15_drifter_wd_y        (obs) float64 -1.452e-06 nan ... 7.493e-08
    e5_cstrio_z15_alti_wd_x           (obs) float64 -3.97e-07 nan ... 1.724e-07
    e5_cstrio_z15_alti_wd_y           (obs) float64 -8.688e-07 nan ... 6.942e-08
    e5_cstrio_z15_drifter_wd_x        (obs) float64 -5.052e-07 nan ... 1.701e-07
    e5_cstrio_z15_drifter_wd_y        (obs) float64 -1.452e-06 nan ... 7.493e-08
Attributes:
    __id:                   CMEMS-L3-SLA-Jason-3__gps
    __time_coverage_end:    2020-05-01T00:00:00
    __time_coverage_start:  2020-01-01T00:00:00

# CREATE COMB

In [36]:
def find_term_list(_ds, wd_x=None, wd_y=None, grad_x=None, grad_y=None, cutoff=None):
    """ Find lists of every possibilities for each momentum equation terms from the dataset
    Parameters
    ----------
    _ds: dataset
            dataset containing colocalisations
    wd_x, wd_y, grad_x, grad_y : list of str
        contains wd_x, wd_y, grad_x or grad_y variable names in the dataset,
        if not provided, will find all possibly corresponding variables in dataset
    cutoff : list of str
            cutoff frequency for drifter variables ('0' for no filter applied)
    
    """
    if not wd_x : wd_x = [l for l in _ds if "wd_x" in l]
    if not wd_y : wd_y = [l for l in _ds if "wd_y" in l]
    if not grad_x : grad_x = [l for l in _ds if "ggx_adt" in l or "ggx_sla" in l]
    if not grad_y : grad_y = [l for l in _ds if "ggy_adt" in l or "ggy_sla" in l]
    if not cutoff : cutoff = [l.split('acc_x_')[-1] for l in _ds if "acc_x" in l]
    return wd_x, wd_y, grad_x, grad_y, cutoff
    
def combinations(_ds, wd_x=None, wd_y=None, grad_x=None, grad_y=None, cutoff=None):
    """Create a list of dictionnaries containing the different data combinations possible to rebuild the moment conservation

    Parameters
    ----------
    _ds: dataset
        contains - drifter_acc_x/y,
                 - drifter_coriolisx/y,
                 - sla gradients from the different sources all fiishing with '_ggx/y',
                 - wind terms from the different sources and way to compute it from stress all finishing with '_wd_x/y'
    wd_x, wd_y, grad_x, grad_y : list of str
        contains wd_x, wd_y, grad_x or grad_y variable names in the dataset we want to consider for combinations
    cutoff : list of str
            cutoff frequency for drifter variables ('0' for no filter applied) we want to consider for combinations
    Returns
    ----------
    [{'acc': 'drifter_acc_x','coriolis': 'drifter_coriolis_x','ggx': 'alti_ggx','wind': 'es_cstrio_z0_alti_wd_x','id': 'co_es_cstrio_z0_alti_x'},....]
    list of dictionnaries containing the varaibles taken for each term and an identification: gradsrc_wdsrc_wdmethod_wddepth_matchupposition_x/y
    """
    
    wd_x, wd_y, grad_x, grad_y, cutoff = find_term_list(_ds, wd_x, wd_y, grad_x, grad_y, cutoff)
    
    LIST = []

    for cf in cutoff :
        for grad in grad_x :
            for wd in wd_x :
                lx = {
                    "acc": 'drifter_acc_x_'+cf,
                    "coriolis": 'drifter_coriolis_x_'+cf,
                    "ggrad": "",
                    "wind": "",
                    "id": "",
                }
                # AVISO grad
                if "aviso" in grad:  #
                    if ("alti" in grad) and ("alti" in wd):
                        lx["ggrad"] = grad
                        lx["wind"] = wd
                        lx["id"] = (
                            "aviso__"
                            +cf+'__'
                            +grad[-3:]
                            +'__'
                            +"_".join(wd.split("_")[:3])
                            + "__alti_x"
                        )
                        LIST.append(lx)
                    elif ("drifter" in grad) and ("drifter" in wd):
                        lx["ggrad"] = grad
                        lx["wind"] = wd
                        lx["id"] = (
                            "aviso__"
                            +cf+'__'
                            +grad[-3:]
                            +'__'
                            + "_".join(wd.split("_")[:3])
                            + "__drifter_x"
                        )
                        LIST.append(lx)
                # Altimeters' grad
                elif "alti" in grad:
                    if "alti" in wd:
                        lx["ggrad"] = grad
                        lx["wind"] = wd
                        lx["id"] = (
                            "co__"
                            +cf+'_'
                            + grad.replace("alti_", "").replace("ggx", "")
                            +'__'
                            + "_".join(wd.split("_")[:3])
                            + "__alti_x"
                        )
                        LIST.append(lx)
                    elif "drifter" in wd:
                        lx["ggrad"] = grad
                        lx["wind"] = wd
                        lx["id"] = (
                            "co__"
                            +cf+'_'
                            + grad.replace("alti_", "").replace("ggx", "")
                            +'__'
                            + "_".join(wd.split("_")[:3])
                            + "__drifter_x"
                        )
                        LIST.append(lx)
        for grad in grad_y:
            for wd in wd_y:
                ly = {
                    "acc": "drifter_acc_y_"+cf,
                    "coriolis": "drifter_coriolis_y_"+cf,
                    "ggrad": "",
                    "wind": "",
                    "id": "",
                }
    
                if ("alti" in grad) and ("alti" in wd):
                    ly["ggrad"] = grad
                    ly["wind"] = wd
                    ly["id"] = (
                        "aviso__"
                        +cf+'__'
                        +grad[-3:]
                        + "__"
                        + "_".join(wd.split("_")[:3])
                        + "__alti_y"
                    )
                    LIST.append(ly)
    
                elif ("drifter" in grad) and ("drifter" in wd):
                    ly["ggrad"] = grad
                    ly["wind"] = wd
                    ly["id"] = (                            
                        "aviso__"
                        +cf+'__'
                        +grad[-3:]
                        + "__"
                        + "_".join(wd.split("_")[:3])
                        + "__drifter_y"
                    )
                    LIST.append(ly)
    return LIST

In [5]:
cluster.close()